In [58]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, RepeatedKFold, cross_val_predict, cross_val_score
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, confusion_matrix

In [59]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import balanced_accuracy_score, confusion_matrix

In [60]:
from sklearn.preprocessing import MinMaxScaler

In [61]:
# на торче
from torch import Tensor
from torch.nn import Sequential, ReLU, Linear, Softmax, BCELoss, Dropout, BatchNorm1d, Module
from torch.nn.init import kaiming_normal_, zeros_
from torch.optim import Adam
import numpy as np

In [62]:
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import CosineAnnealingLR

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [65]:
data = pd.read_csv('rakota.csv')
validation = pd.read_csv('rakota2.csv')

In [66]:
Y = data['CANCER']
X = data.loc[:, ['AGE', 'CA125', 'HE4']]

In [67]:
Yv = validation['CANCER'] 
Xv = validation.loc[:, ['AGE', 'CA125', 'HE4']] 

In [68]:
rs = RandomOverSampler()
Xr, Yr = rs.fit_resample(X, Y)

In [69]:
mms = MinMaxScaler()

In [70]:
Xn = mms.fit_transform(Xr)

In [71]:
Xten = Tensor(Xn)

In [72]:
Xvn = mms.transform(Xv)

In [73]:
Xvten = Tensor(Xvn)

In [74]:
Ybin = np.vstack([~Yr.astype(bool), Yr]).T
Yten = Tensor(Ybin)

In [75]:
dl = DataLoader(TensorDataset(Xten, Yten), batch_size=10, shuffle=True)

In [76]:
l1 = Linear(3, 50)
l2 = Linear(50, 50)
l3 = Linear(50, 2)
kaiming_normal_(l1.weight)
kaiming_normal_(l2.weight)
kaiming_normal_(l3.weight)
zeros_(l1.bias)
zeros_(l2.bias)
zeros_(l3.bias)

Parameter containing:
tensor([0., 0.], requires_grad=True)

In [77]:
mlpc = Sequential(l1, ReLU(), l2, ReLU(), l3, Softmax(1)) #MLPClassifier
mlpc

Sequential(
  (0): Linear(in_features=3, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=2, bias=True)
  (5): Softmax(dim=1)
)

In [78]:
loss = BCELoss()
optim = Adam(mlpc.parameters() ,lr=.1)
sched = CosineAnnealingLR(optim, 50, .0001)

In [79]:
for e in range(50):
    for Xb, Yb in dl:
        pred = mlpc(Xb)
        l = loss(pred, Yb)
        optim.zero_grad()
        l.backward()
        optim.step()
    sched.step()

In [80]:
pred = mlpc(Xvten)
Yv_pred = (pred[:, 1] > .5).numpy()

In [81]:
confusion_matrix(Yv, Yv_pred)

array([[118,  17],
       [  0,  24]])

In [ ]:
balanced_accuracy_score(Yv, Yv_pred)